Day-1：线性神经网络
线性神经网络是只有一个单层神经网络，虽然只有一个隐含层，但线性神经网络的训练流程，大体上与体积更庞大的神经网络一致。本节重点讨论的是梯度如何在神经网络中传播，以及如何使用梯度下降法对神经网络参数进行调整。
首先导入所需的库，由于需要记录变量的梯度信息，因此使用torch.tensor数据结构代替numpy

In [3]:
import random
import torch as pt
import numpy as np
from tqdm import tqdm

class linear_regression():
    def __init__(self,batch_size=None,learning_rate=None,
                 init_alpha:pt.tensor=None,init_beta:pt.tensor=None,epoch=None):

        self.batch_size=batch_size
        self.lr=learning_rate

        self.alpha=init_alpha
        self.alpha.requires_grad_(requires_grad = True)

        self.beta=init_beta
        self.beta.requires_grad_(requires_grad=True)

        self.epoch=epoch

    def generate_test_data(self,x_dim):
        alpha=pt.tensor((np.arange(x_dim)+2),dtype=pt.float32).unsqueeze(1)
        beta = pt.tensor(2,dtype=pt.float32)
        x=pt.randn(4000,len(alpha),dtype=pt.float32)
        y=pt.mm(x,alpha)+beta
        y=y+pt.tensor(np.random.normal(0,1,size=y.size()),dtype=pt.float32)
        print('vector_alpha=',alpha,'\n')
        print('scalar_beta=',beta,'\n')
        return x,y

    def generate_train_data(self,x,y):
            num_examples = len(x)
            indices = list(range(num_examples))
            random.shuffle(indices)  # 样本的读取顺序是随机的
            for i in range(0, num_examples, self.batch_size):
                j = pt.LongTensor(indices[i: min(i + self.batch_size, num_examples)])  # 最后一次可能不足一个batch
                yield x.index_select(0, j), y.index_select(0, j)

    def linear_model(self,x):
        value=pt.mm(self.alpha.unsqueeze(1).T,x.T)+self.beta
        return value

    """我们计算loss是对某个batch计算loss
        在定义输入数据时，我们就要对数据的格式进行要求
        行数对应变量序号，列对应各个变量     
                                    """
    def loss_function(self,y_hat,y):
        loss = (y_hat - y.view(y_hat.size()))**2/ 2
        return loss

    """此处使用的是mini-batch sdg
    在训练时对batch内的样本逐一计算损失，全部输出后将损失累加
    对累加的损失计算一个全局的梯度，随后在优化时，除batch-size得到batch平均梯度
    这部分如何编写仍存在疑义"""
    def sgd(self):
        #alpha是向量，反向传播的梯度与alpha形状一致，也是向量
        self.alpha.data = self.alpha.data - self.alpha.grad*self.lr/self.batch_size
        self.beta.data = self.beta.data - self.beta.grad * self.lr / self.batch_size


    def train_linear_model(self,x,y):
        for i in tqdm(range(self.epoch)):
            for train_data,train_value in self.generate_train_data(x,y):
                loss = self.loss_function(self.linear_model(train_data),train_value).sum()
                loss.backward()
                self.sgd()
                self.alpha.grad.data.zero_()
                self.beta.grad.data.zero_()
            loss_all_data = self.loss_function(self.linear_model(x), y).mean().item()
            
        print('loss=',loss_all_data)

本节所编写的线性神经网络，是基于面对对象的思路编写的。
首先我们定义linear_regression对象，并设置对象的初始化函数。
初始化函数的功能，一方面初始化网络超参数（batch_size,learning_rate,epoch)；一方面初始化网络的参数(alpha,beta)。
由于我们编写的是线性神经网络，网络的参数模式是确定的(ax+b=y)。
注意，在设置网络参数时，需要开启tensor类型的梯度记录功能，开启后网络将记录tensor内变量参与的梯度计算。

第二步我们定义生成测试数据的函数。线性神经网络，是一个没有激活函数的单层神经网络。神经网络具有拟合数据的特性，因此线性神经网络可以拟合任何的线性函数f(x)。此处我们利用一个参数确定的线性函数生成具有线性关系的x与y。

此处根据预先设置的batch_size设置训练组，本质是一个从训练集中随机取样生成固定大小训练组的过程。这一步在现有的深度学习框架中由dataloader函数完成。

在设置好训练数据后，我们设置线性模型。设置的模型，本质上是设置一个计算流程，需要在流程中定义由输入的x如何计算输出y。
这一步在现有的深度学习框架中有不同的实现方法，但原理相同。

神经网络的设置流程大致为：
准备训练数据，设置模型（x与y的预期对应关系），设置模型损失函数（如何度量偏差）。
对于线性神经网络，我们设置的偏差为均方误差。
注意，此处的均方误差计算函数（loss_function)，是针对某“一个”数据x，生成的估计值y_hat与其对应训练值y计算的。
在下一步参数优化中，将解释为什么损失是逐个向量计算。

梯度下降法，是每次将变量依据其梯度方向变化一定步长的过程。梯度下降法具有较多的衍生形态。
其中，随机梯度下降法，每计算一次梯度就会变化一次参数。
本次使用的mini_batch stochastic gradient descent ，相较于SGD，每计算一个mini_batch组后，才会在这一组梯度的平均方向上进行一次下降。

设置数据集，模型与损失函数后，我们使用定义的Mini_bitch SGD算法对模型参数进行优化。
由于使用的是Mini_batch SGD ，参数变化的梯度方向为每个mini batch 梯度方向的平均值
在编程中，mini batch SGD的关键点体现在SGD函数中学习率除以batch_size，与训练函数中，计算损失后使用.sun()进行梯度累加（求和，平均）。
注意，在每论训练后，需要清零参数的梯度值

In [2]:
if __name__ == "__main__":
    model_1=linear_regression(batch_size=10,epoch=200,learning_rate=0.0001,
                              init_alpha=pt.tensor([1,1,1],dtype=pt.float32),
                              init_beta=pt.tensor([1],dtype=pt.float32))
    x,y=model_1.generate_test_data(x_dim=3)
    model_1.train_linear_model(x,y)
    print(model_1.alpha)
    print(model_1.beta)

vector_alpha= tensor([[2.],
        [3.],
        [4.]]) 

scalar_beta= tensor(2.) 



100%|██████████| 200/200 [00:19<00:00, 10.43it/s]

loss= 0.5209633708000183
tensor([1.9818, 2.9839, 4.0006], requires_grad=True)
tensor([1.9972], requires_grad=True)
